S VS N CLASSIFICATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/BERT_data/'

df = pd.read_csv(binary_data_path + "N_vs_S_data_withEmoji.csv")
df = df.dropna(how='any',axis=0)
df.sample(10)

,label,text
148849,1,'my best male friend enfp people call 'best fr...
94246,0,drink occasionally high chance alcoholic used ...
8494,0,similar people like think funny smart kind wha...
55546,0,intj intp easily natural though socially adapt...
177003,1,least likely w
128386,1,congratulations dear
79952,0,several dates intj wow wait long intj date ano...
31089,0,oh nuance currently existing russian socionic ...
110783,1,days paranoia healthy dose mistrust forms hosp...
162742,1,lol joked


In [9]:
DOWNSAMPLE = 30000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

0    15041
1    14959
Name: label, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [11]:
train.head(3)

,label,text
140791,1,l evil still intp light enxj ryuk istp
1449,0,never talk people behind backs hear people sti...
152150,1,original poster asking feedback regarding exce...


In [12]:
test.head(3)

,label,text
144436,1,know best-friend lets saay never alone ^^
33890,0,meant find home ca loved anyone still chasing ...
93168,0,like japanese/samurai-themed shows check lone ...


In [14]:
ourpur_dir = binary_data_path + "SvsN_BERT_withEmojis/"
train.to_csv(ourpur_dir+"S_vs_N_train_data.csv", index=False)
test.to_csv(ourpur_dir+"S_vs_N_test_data.csv", index=False)

In [15]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 72.6 MB/s 
     |████████████████████████████████| 182 kB 63.1 MB/s 
     |████████████████████████████████| 132 kB 71.9 MB/s 
     |████████████████████████████████| 127 kB 75.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [16]:
data_files = {"train": ourpur_dir+"S_vs_N_train_data.csv", "test": ourpur_dir+"S_vs_N_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')



Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0dab0685abebe0e1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4500
    })
})

In [18]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 77.3 MB/s 


In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [20]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [21]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

4500

In [22]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/25500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

In [23]:
tokenized_train[0]

{'label': 0,
 'text': 'quad-shot ahaha',
 'input_ids': [101, 17718, 1011, 2915, 6289, 23278, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [24]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [26]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [27]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [28]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7970
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss
500,0.691400
1000,0.672200
1500,0.668000
2000,0.649500
2500,0.635100
3000,0.624800
3500,0.577100
4000,0.540600
4500,0.534000
5000,0.496300


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/SvsN_BERT_withEmojis/checkpoint-1000

TrainOutput(global_step=7970, training_loss=0.5239335638067804, metrics={'train_runtime': 313.9601, 'train_samples_per_second': 406.103, 'train_steps_per_second': 25.385, 'total_flos': 991156263396960.0, 'train_loss': 0.5239335638067804, 'epoch': 5.0})

In [29]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 16


<ipython-input-26-b1430cee0b64>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.9391549229621887,
 'eval_accuracy': 0.6131111111111112,
 'eval_f1': 0.6112971645456575,
 'eval_runtime': 4.5712,
 'eval_samples_per_second': 984.425,
 'eval_steps_per_second': 61.691,
 'epoch': 5.0}